In [1]:
# this code will find your face, overlay an image on your face, and record a video of you with the filter on

import cv2
import os
import numpy as np

# opencv classifier for face and eyes detection
cascPathface = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
cascPatheyes = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_eye_tree_eyeglasses.xml"

faceCascade = cv2.CascadeClassifier(cascPathface) # initialize face classifier
eyeCascade = cv2.CascadeClassifier(cascPatheyes)  # initialize eyes classifier

cartoon = cv2.imread("cartoon.jpg") # image we are going to put over our face

video_capture = cv2.VideoCapture(0) # turn our camera on

width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))    # get camera resolution
height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
filename = "output.avi"                                     # name of video
fps = 24                                                    # FPS of video
fourcc = cv2.VideoWriter_fourcc(*"XVID")                    # video encoder
out=cv2.VideoWriter(filename, fourcc, fps, (width, height)) # create output video

while True:
    
    # Get video feed
    
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Get face and eyes
    
    # find faces
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE) 
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h),(0,255,0), 2) # outline faces
        faceROI = frame[y:y+h,x:x+w]                              # find just the face without background
        eyes = eyeCascade.detectMultiScale(faceROI)               # find eyes from just the face frame, not webcam frame
        for (x2, y2, w2, h2) in eyes:                             # outline eyes
            eye_center = (x + x2 + w2 // 2, y + y2 + h2 // 2)
            radius = int(round((w2 + h2) * 0.25))
            frame = cv2.circle(frame, eye_center, radius, (255, 0, 0), 4)
    
    # Create Filter
    
    # Reshape cartoon face to the shape of faceROI
    cartoon_resized = cv2.resize(cartoon, (int(faceROI.shape[1]),int(faceROI.shape[0])), interpolation = cv2.INTER_AREA)
    
    # Replace face ROI with cartoon
    background = frame.copy()             # define back/foreground so everything is in real time rather than 
    foreground = cartoon_resized.copy()   # on 1 frame that the filter paints over, you will get it if you don't do this
    x_offset= x
    y_offset= y
    # Impose shifted/offsetted foreground onto background 
    background[y_offset:y_offset+foreground.shape[0], x_offset:x_offset+foreground.shape[1]] = foreground 
    
    # Record Vid
    
    # ret (bool return value on webcam read function), so if we can read the webcam properly
    if ret == True:
        out.write(background)

    # Display the resulting frame
    cv2.imshow('Face Video | Filter', np.hstack([frame, background]))
    cv2.imshow("Face ROI", faceROI)
    
    # press q to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
video_capture.release() # relases access to camera
cv2.destroyAllWindows() # closes all related windows